<a href="https://colab.research.google.com/github/s-kamireddy/Logistic-Regression-for-Heart-Attack-Risk-Predictiom/blob/main/projects_in_ai_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required libraries from titanic example
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import Normalizer
import torch
import warnings
warnings.filterwarnings("ignore")

Part 1:
1. Derive the objective function for Logistic Regression using Maximum Likelihood
Estimation (MLE). Do some research on the MAP technique for Logistic Regression,
include your research on how this technique is different from MLE (include citations).

  >Deriving the logit function:

  Starting with the liklihood function:

  $L(w) = Π_{y_i = 1}(p(x^i)) * Π_{y_i = 1}(1-p(x^i)) $

  $= Π(p(x^i)^{y^i} + (1-p(x^i))^{1-y^i})$

  Our goal is to minimize this function, the minimum of the log will occur at the same place so we can take the logarithm and divide by the number of samples:

  $l(w) =\frac{1}{n} log(Π(p(x^i)^{y^i} + (1-p(x^i))^{1-y^i}))$

  $ = \frac{1}{n}Σ( y^i log(p(x^i)) + (1-y^i)log(1-p(x^i)) )$

  $p(x^i) = \frac{1}{1+ e^{-wx^i}} $

  $l(w) = \frac{1}{n}Σ( y^i log( \frac{1}{1+ e^{-wx^i}}) + (1-y^i)log(1-  \frac{1}{1+ e^{-wx^i}}) )$

  $ = \frac{1}{n}Σ( y^i log( \frac{1}{1+ e^{-wx^i}}) + (1-y^i)log( \frac{e^{-wx^i}}{1+ e^{-wx^i}}) )$

  $= \frac{1}{n}Σ( y^i (log( \frac{1}{1+ e^{-wx^i}}) - log(\frac{e^{-wx^i}}{1+ e^{-wx^i}}) ) + log( \frac{e^{-wx^i}}{1+ e^{-wx^i}}) )$
  $= \frac{1}{n}Σ( y^i (log( \frac{1}{1+ e^{-wx^i}}) - log(\frac{e^{-wx^i}}{1+ e^{-wx^i}}) ) ) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}})  $
  $ =  \frac{1}{n}Σ( y^i (log( \frac{1}{1+ e^{-wx^i}}) - log(\frac{1}{1+ e^{-wx^i}}) + log(e^{-wx^i})) ) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}}) = \frac{1}{n}Σ( y^i ( log(e^{-wx^i})) ) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}})  $
  $=  \frac{1}{n}Σ( -y^iwx^i) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}})$

  which is our objective function

2. Define a machine learning problem you wish to solve using Logistic Regression. Justify
why logistic regression is the best choice and compare it briefly to another linear
classification model (cite your work if this other technique was not covered in class).

The machine learning problem I would like to solve using logistic regression is the liklihood of a heart attack based on a number of demographic and lifestyle factors. This is because the values are binary (heart attack vs. no heart attack) and we want a liklihood as the output of learning (between 0 and 1) making logistic regression a good choice compared to something like linear regression can output any number in the space of real numbers.

3. Discuss how your dataset corresponds to the variables in your equations, highlighting
any assumptions in your derivation from part 1.

Assuming x is [1, x_1, x_2, ... , x_d] and w is of the form [w_0, w_1, w_2, ..., w_d] (where w_0 is the bias and the rest of the w values correspond to weights). $x^i$ corresponds to the lifestyle factors of each individual




Part 2:
----
1. Link to dataset: https://www.kaggle.com/datasets/iamsouravbanerjee/heart-attack-prediction-dataset


In [ ]:
#2. EDA
df = pd.read_csv('https://raw.githubusercontent.com/s-kamireddy/Logistic-Regression-for-Heart-Attack-Risk-Predictiom/refs/heads/main/heart_attack_prediction_dataset.csv?token=GHSAT0AAAAAAC5KVJTC2A4UXR6X2L2X7E56Z4PYHVA')

HTTPError: HTTP Error 404: Not Found

**Data Dictionary**



In [ ]:
df.head()

In [ ]:
df.shape
df.isna().sum()


In [ ]:
df.info()

In [ ]:
df.describe() #dataset stats

In [ ]:
df.describe(include=['O'])

In [ ]:
df["Heart Attack Risk"].value_counts(normalize=True)

In [ ]:

# Convert categorical data to numerical data using cat.codes
df['Sex'] = df['Sex'].astype('category')
df['Sex'] = df['Sex'].cat.codes
df['Exercise Hours Per Week'] = df['Exercise Hours Per Week'].astype('category')
df['Exercise Hours Per Week'] = df['Exercise Hours Per Week'].cat.codes
df['Diet'] = df['Diet'].astype('category')
df['Diet'] = df['Diet'].cat.codes
df['Country'] = df['Country'].astype('category')
df['Country'] = df['Country'].cat.codes
df['Continent'] = df['Continent'].astype('category')
df['Continent'] = df['Continent'].cat.codes
df['Hemisphere'] = df['Hemisphere'].astype('category')
df['Hemisphere'] = df['Hemisphere'].cat.codes

def bp_value(str):
  str = str.split('/')
  return (float(str[0])+ (2*float(str[1])))/3

df['Blood Pressure'] = df['Blood Pressure'].map(bp_value)

#remove patient ID
df.drop('Patient ID', axis=1, inplace=True)



In [ ]:
#sns.pairplot(df, hue = 'Heart Attack Risk')

In [ ]:
def logit_cost(X, Y, w):
  cost = 0
  cost_function = lambda x, y, w: (torch.exp(-torch.dot(x, w))/(1+torch.exp(-torch.dot(x, w)))) - y*torch.dot(x,w)
  for i in range(len(X)):
    x = X[i]
    y = Y[i]
    cost += cost_function(x, y, w)
  return cost/len(X)


In [ ]:
def gradient_descent(w, X, Y):
  for _ in range(1000):
    gradient = torch.zeros.shape(w)
    for i in range(len(X)):
      x = X[i]
      y = Y[i]
      gradient += (y * x)/(1 + torch.exp(y * torch.dot(x, w)))
    gradinet /= len(X)
    w -= gradient * 0.001



  return w


In [ ]:
Y = df['Heart Attack Risk'].values
X = df.drop('Heart Attack Risk', axis=1).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
x_train = torch.tensor(X_train)
y_train = torch.tensor(Y_train)
x_test = torch.tensor(X_test)
y_test = torch.tensor(Y_test)